# Import & config

In [721]:
import requests
import time
import os
import re
import pandas as pd
from bs4 import BeautifulSoup

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36'}

### 1. Get all countries

In [722]:
def get_countries():
  url = "https://www.numbeo.com/cost-of-living/"
  res = requests.get(url, headers=headers)
  soup = BeautifulSoup(res.content, 'html.parser')

  options = soup.select('#country > option')

  countries = []
  
  for option in options:
    if option.text != "---Select country---":
      countries.append(option.text)
  
  return countries

### 2. Extract data

In [723]:
def extract_cost_of_living(currency, location):
  url = f"https://www.numbeo.com/cost-of-living/country_result.jsp?country={location.replace(" ", "+")}&displayCurrency={currency}"

  print(url)

  res = requests.get(url, headers=headers)
  soup = BeautifulSoup(res.content, 'html.parser')

  table = soup.select('table.data_wide_table.new_bar_table > tr:has(.priceValue)')

  locations = []
  names = []
  prices = []

  for tr in table:
    for row in tr.contents:
      if row.text == " ":
        tr.contents.remove(row)

    name = tr.contents[0].text.strip()
    price = tr.contents[1].text.replace("\xa0€", "").strip()

    locations.append(location)
    names.append(name)
    prices.append(price)
  
  return locations, names, prices

In [724]:
def create_csv():

  countries = get_countries()

  d = { "Country": [], "Name": [], "Price": [] }

  for country in countries[0:2]:
    data = extract_cost_of_living("USD", country)

    country_names = data[0]
    items = data[1]
    prices = data[2]

    d["Country"].extend(country_names)
    d["Name"].extend(items)
    d["Price"].extend(prices)


  df = pd.DataFrame(data=d)
  df.to_csv(os.path.join('data/', 'living_coast.csv'), index=False)



In [725]:
create_csv()

https://www.numbeo.com/cost-of-living/country_result.jsp?country=Afghanistan&displayCurrency=USD


/tmp/ipykernel_3819/1071666125.py:7: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(res.content, 'html.parser')


https://www.numbeo.com/cost-of-living/country_result.jsp?country=Aland+Islands&displayCurrency=USD
